# 基于隐马尔可夫模型（HMM）的金融市场状态识别

## 项目目标
应用HMM对S&P 500指数的日收益率进行建模，识别出市场背后潜在的宏观状态（如“牛市”、“熊市”、“震荡市”），并对状态的特征和转移规律进行分析。

### 步骤一：数据准备与探索

In [2]:
# 导入所需库
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from hmmlearn import hmm

# 设置图表样式
sns.set_style('whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

In [ ]:
# 获取数据：S&P 500 ETF (SPY)
start_date = '2000-01-01'
end_date = '2023-12-31'
ticker = 'SPY'

# 下载数据
# yfinance v0.2.x 默认返回调整后的价格在'Close'列中
data = yf.download(ticker, start=start_date, end=end_date)

# 计算每日对数收益率
data['Log_Return'] = np.log(data['Close'] / data['Close'].shift(1))

# 删除第一个NaN值
data.dropna(inplace=True)

# 将对数收益率序列作为观测序列
returns = data['Log_Return'].values.reshape(-1, 1)

print(f"获取了 {len(data)} 天的数据。")
print("数据预览：")
data.head()

C:\Users\83774\AppData\Local\Temp\ipykernel_25732\196640936.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


KeyError: 'Adj Close'

In [ ]:
# 绘制收盘价和对数收益率
fig, ax = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

# 绘制价格走势
ax[0].plot(data.index, data['Close'], label='SPY收盘价', color='blue')
ax[0].set_title('S&P 500 ETF (SPY) 价格走势 (2000-2023)')
ax[0].set_ylabel('价格 (美元)')
ax[0].legend()

# 绘制对数收益率
ax[1].plot(data.index, data['Log_Return'], label='每日对数收益率', color='green')
ax[1].set_title('S&P 500 ETF (SPY) 每日对数收益率 (2000-2023)')
ax[1].set_ylabel('对数收益率')
ax[1].legend()

plt.tight_layout()
plt.show()

### 步骤二：模型构建与训练

In [ ]:
# 创建并训练高斯HMM
# 我们设定3个隐藏状态，分别对应“牛市”、“熊市”和“震荡市”
n_components = 3
model = hmm.GaussianHMM(n_components=n_components, covariance_type="full", n_iter=1000, random_state=42)

# 使用对数收益率序列训练模型
model.fit(returns)

print("模型训练完成。")

In [ ]:
# 打印学习到的参数，进行初步检查
print("状态转移概率矩阵 (A):")
print(np.round(model.transmat_, 4))

print("\n每个隐藏状态下高斯分布的均值 (Means):")
print(np.round(model.means_, 6))

print("\n每个隐藏状态下高斯分布的协方差 (Covariances):")
# 我们关心的是波动率，即标准差的平方
for i in range(n_components):
    print(f"状态 {i} 的日波动率 (标准差): {np.sqrt(model.covars_[i][0][0]):.6f}")

### 步骤三：模型解码与结果分析

In [ ]:
# 使用维特比算法解码，得到最有可能的隐藏状态序列
hidden_states = model.predict(returns)

print("成功为每一天预测了隐藏状态。")
print("前10天的状态是:", hidden_states[:10])

#### 状态解读与命名

为了方便解读，我们根据每个状态的统计特性（均值和波动率）为其命名。通常：

- **牛市 (Bull Market)**: 均值为正，波动率较低。
- **熊市 (Bear Market)**: 均值为负，波动率极高。
- **震荡/平稳市 (Neutral/Sideways Market)**: 均值接近零，波动率中等。

下面的代码将根据均值对状态进行排序和命名。

In [ ]:
# 根据均值对状态进行排序，以便命名
# argsort() 返回的是从小到大的索引
sorted_means_idx = np.argsort(model.means_.flatten())

# 创建一个从旧索引到新有序索引的映射
# 例如，如果均值最低的状态是2，均值中间的是0，最高的是1，则 sorted_means_idx = [2, 0, 1]
# 我们希望将状态2命名为“熊市”(0)，状态0为“震荡市”(1)，状态1为“牛市”(2)
state_map = {old_idx: new_idx for new_idx, old_idx in enumerate(sorted_means_idx)}

# 状态名称
state_names = ["熊市", "震荡市", "牛市"]

# 重新标记隐藏状态
remapped_states = np.array([state_map[s] for s in hidden_states])

# 将预测的状态添加到数据框中
data['State'] = remapped_states
data['State_Name'] = data['State'].apply(lambda s: state_names[s])

print("状态命名和重新标记完成。")
data.head()

#### 核心产出1：参数分析表

In [ ]:
# 创建参数分析表
params_df = pd.DataFrame()

for i, state_idx in enumerate(sorted_means_idx):
    state_name = state_names[i]
    mean = model.means_[state_idx][0]
    std_dev = np.sqrt(model.covars_[state_idx][0][0])
    proportion = np.sum(hidden_states == state_idx) / len(hidden_states)
    
    params_df = pd.concat([params_df, pd.DataFrame({
        '状态名称': [state_name],
        '日均收益率': [mean],
        '日波动率 (标准差)': [std_dev],
        '时间占比': [f"{proportion:.2%}"]
    })], ignore_index=True)

params_df.set_index('状态名称', inplace=True)

print("核心产出1：参数分析表")
params_df

#### 核心产出2：状态转移概率矩阵

In [ ]:
# 提取原始转移矩阵
original_transmat = model.transmat_

# 根据排序后的索引重新排列转移矩阵
# sorted_means_idx 包含了从新状态到旧状态的映射关系
# 例如, sorted_means_idx = [2, 0, 1] 意味着新状态0(熊市)是旧状态2，新状态1(震荡)是旧状态0，新状态2(牛市)是旧状态1
remapped_transmat = np.zeros_like(original_transmat)

for i, old_row_idx in enumerate(sorted_means_idx):
    for j, old_col_idx in enumerate(sorted_means_idx):
        remapped_transmat[i, j] = original_transmat[old_row_idx, old_col_idx]

# 创建DataFrame以便更好地展示
transmat_df = pd.DataFrame(remapped_transmat, columns=state_names, index=state_names)

print("核心产出2：状态转移概率矩阵")
transmat_df

#### 矩阵解读

该矩阵展示了从一个状态（行）转移到另一个状态（列）的概率。

- **对角线元素**：代表市场状态的“粘性”或“持续性”。值越高，意味着市场越倾向于保持在当前状态。
- **非对角线元素**：代表市场状态发生转换的概率。例如，第一行第二列的数值代表了从“熊市”转移到“震荡市”的概率。

#### 核心产出3（图形）：带状态标记的价格走势图

这幅图将S&P 500指数的价格走势与模型识别出的市场状态结合在一起。背景颜色代表了每一天的市场状态：

- **绿色**: 牛市
- **红色**: 熊市
- **灰色**: 震荡市

我们还将标注出几个重大的历史经济事件，以检验模型识别出的“熊市”是否与真实世界的危机时刻相对应。

In [ ]:
# 创建图形
fig, ax = plt.subplots(figsize=(20, 10))

# 定义颜色和标签
state_colors = {0: 'red', 1: 'grey', 2: 'green'}
state_labels = {0: '熊市', 1: '震荡市', 2: '牛市'}

# 为了图例只显示一次，我们跟踪已添加的标签
plotted_labels = set()

# 遍历每个状态，并为连续的时间段着色
for state_code, state_name in state_labels.items():
    # 找出该状态对应的所有日期
    state_dates = data.index[data['State'] == state_code]
    
    # 找出连续时间段的起始点
    # 如果当前日期与前一个日期的间隔超过一个交易日，则认为是一个新时间段的开始
    block_starts = [state_dates[0]] + [d for i, d in enumerate(state_dates[1:]) if (d - state_dates[i]).days > 3]

    for start_date in block_starts:
        # 找到这个连续时间段的结束点
        end_date = start_date
        temp_date = start_date
        while True:
            next_day = temp_date + pd.Timedelta(days=1)
            if next_day in state_dates:
                end_date = next_day
                temp_date = next_day
            else:
                break
        
        label = state_name if state_name not in plotted_labels else None
        ax.axvspan(start_date, end_date, color=state_colors[state_code], alpha=0.3, label=label)
        if label:
            plotted_labels.add(state_name)

# 绘制价格曲线
ax.plot(data.index, data['Adj Close'], color='black', label='SPY 调整后收盘价', linewidth=1.0)

# 标注重要经济事件
ax.axvline(pd.to_datetime('2008-09-15'), color='purple', linestyle='--', linewidth=2, label='雷曼兄弟破产 (2008)')
ax.axvline(pd.to_datetime('2020-03-09'), color='orange', linestyle='--', linewidth=2, label='新冠疫情市场熔断 (2020)')

# 设置图表属性
ax.set_title('S&P 500 市场状态识别 (2000-2023)', fontsize=20)
ax.set_ylabel('价格 (美元)', fontsize=14)
ax.set_xlabel('日期', fontsize=14)
ax.legend(loc='upper left', fontsize=12)
ax.grid(False)

plt.show()

## 项目总结

本项目成功应用了高斯隐马尔可夫模型（GaussianHMM）对标准普尔500指数2000年至2023年的日收益率数据进行了建模，旨在识别金融市场背后不可见的宏观状态。

**主要发现：**

1.  **状态识别**：模型成功识别出三个具有显著不同特征的隐藏状态，我们根据其日均收益率和波动率分别命名为“牛市”、“熊市”和“震荡市”。
2.  **状态特征**：
    - “牛市”状态具有稳定的正收益和低波动性。
    - “熊市”状态表现为显著的负收益和极高的波动性。
    - “震荡市”则介于两者之间，收益接近于零，波动性中等。
3.  **状态转移**：通过分析状态转移矩阵，我们发现市场状态具有很强的“粘性”，即倾向于保持在当前状态。从“熊市”或“牛市”直接转换到对方的概率非常低，通常需要经过“震荡市”作为过渡。
4.  **模型验证**：最终的可视化结果显示，模型识别出的“熊市”阶段与2008年全球金融危机和2020年新冠疫情引发的市场暴跌高度吻合，有力地验证了模型的有效性和解释力。

综上所述，隐马尔可夫模型是一个强大的工具，能够有效地从嘈杂的金融时间序列中提取出有意义的宏观市场结构，为投资决策和风险管理提供了有价值的洞察。